# Write SEDs to the VO

Loop over all the photo-z 'best' SEDs in dmu28 and write them all to the appropriate VO folders with the final published columns

In [1]:
from  herschelhelp_internal  import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
017bb1e (Mon Jun 18 14:58:59 2018 +0100)
This notebook was executed on: 
2018-06-27 17:34:31.287600


In [26]:
from astropy.table import Table, Column
from astropy import units as u
import numpy as np

import re

import yaml

import glob

from shutil import copyfile


import os
import time

import humanfriendly

In [3]:
TODAY = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))

## Convert the SEDs
Here we define a function to take the SED fits file as output by CIGALE and change it to our final published format. This means replacing the columns thus:


| Output column | relation to input columns                                     |
|---------------|---------------------------------------------------------------|
| WAVE          | wavelength                                                    |
| FLUX          | Fnu                                                           |
| LUMIN         | L_lambda_total                                                |
| STELLAR       | stellar.old + stellar.young                                   |
| DUST          | dust.Umin_Umin + dust.Umin_Umax                               |
| AGN           | agn.fritz2006_therm + agn.fritz2006_scatt + agn.fritz2006_agn |
| IGM           | igm                                                           |

In [32]:
def cigale_to_vo(in_sed_name, out_folder):
    """Convert CIGALE output SED to final VO format
    
    Parameters
    ----------
    in_sed_name: string
        Link to input SED fits file containg astropy Table
        
    Returns
    -------
    out_sed_name: string
        Link to output SED fits file that was written during function run.
    """
    in_sed = Table.read(in_sed_name)
    out_sed_name = out_folder + in_sed_name.split('/')[-1]
    
    out_sed = Table()
    
    out_sed.add_column(Column(data = in_sed['wavelength'], 
                                name = 'WAVE'))
    out_sed.add_column(Column(data = in_sed['Fnu'], 
                                name = 'FLUX'))
    out_sed.add_column(Column(data = in_sed['L_lambda_total'], 
                                name = 'LUMIN'))
    out_sed.add_column(Column(data = in_sed['stellar.old'] + in_sed['stellar.young'], 
                                name = 'STELLAR'))
    out_sed.add_column(Column(data = in_sed['dust.Umin_Umin'] + in_sed['dust.Umin_Umax'], 
                                name = 'DUST'))
    out_sed.add_column(Column(data = in_sed['agn.fritz2006_therm'] + in_sed['agn.fritz2006_scatt'] + in_sed['agn.fritz2006_agn'], 
                                name = 'AGN'))
    out_sed.add_column(Column(data = in_sed['igm'], 
                                name = 'IGM'))
    out_sed.write(out_sed_name, overwrite=True)
    
    #Also try to copy the png
    try:
        copyfile(in_sed_name.replace('.fits', '.png'), 
                 out_sed_name.replace('.fits', '.png'))

    except FileNotFoundError:
        pass
        
    
    return out_sed_name

In [33]:
dmu32_ymls = glob.glob('../dmu32/*/meta_main.yml', recursive=True)

In [41]:
out_folder = '/mnt/hedam/data-vo/bestseds/{}/'
#out_folder = '/Users/rs548/GitHub/gavo_inputs/bestseds/data/{}/'

for yml_file in dmu32_ymls:

    dmu32_yml = yaml.load(open(yml_file, 'r'))
    field = dmu32_yml['field']
    cigale = dmu32_yml['cigale']
    if cigale != 'None':
        print(cigale)
        try:
            os.mkdir(out_folder.format(field))
        except FileExistsError:
            pass
        
        sed_folder = re.sub('/*final_results.fits', '/SEDs/fits/', cigale)
        sed_folder = re.sub('dmu_products', '..', sed_folder)
        sed_folder = re.sub('/HELP_', '', sed_folder)
        print(sed_folder)
        all_seds = glob.glob(sed_folder + '*.fits')
        print(all_seds)
        for in_sed_name in all_seds:
            cigale_to_vo(in_sed_name, out_folder.format(field))
  

dmu_products/dmu28/dmu28_COSMOS/data/zphot/final_results.fits
../dmu28/dmu28_COSMOS/data/zphot/SEDs/fits/
[]
dmu_products/dmu28/dmu28_ELAIS-S1/data/zphot/HELP_final_results.fits
../dmu28/dmu28_ELAIS-S1/data/zphot/SEDs/fits/
[]
dmu_products/dmu28/dmu28_GAMA-09/data/zphot/HELP_final_results.fits
../dmu28/dmu28_GAMA-09/data/zphot/SEDs/fits/
[]
dmu_products/dmu28/dmu28_Lockman-SWIRE/data/zphot/best_extcor/HELP_final_results.fits
../dmu28/dmu28_Lockman-SWIRE/data/zphot/best_extcor/SEDs/fits/
[]
dmu_products/dmu28/dmu28_GAMA-12/data/zphot/HELP_final_results.fits
../dmu28/dmu28_GAMA-12/data/zphot/SEDs/fits/
[]
dmu_products/dmu28/dmu28_GAMA-15/data/zphot/HELP_final_results.fits
../dmu28/dmu28_GAMA-15/data/zphot/SEDs/fits/
[]
dmu_products/dmu28/dmu28_CDFS-SWIRE/data/zphot/HELP_final_results.fits
../dmu28/dmu28_CDFS-SWIRE/data/zphot/SEDs/fits/
[]
dmu_products/dmu28/dmu28_ELAIS-N1/data/zphot/HELP_final_results.fits
../dmu28/dmu28_ELAIS-N1/data/zphot/SEDs/fits/
['../dmu28/dmu28_ELAIS-N1/data/zphot